# Table of Contents
 <p><div class="lev2 toc-item"><a href="#bitrex-code-wip" data-toc-modified-id="bitrex-code-wip-01"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>bitrex code wip</a></div>

In [17]:
import json
import hmac
import time
import base64
import hashlib
import requests
import pandas as pd
from requests.auth import AuthBase

In [18]:
import urllib3

In [19]:
api_cred = pd.read_json("../API_KEYS/api_keys.json", orient= "index")
api_url = pd.read_json("../API_KEYS/api_url.json", orient= "index")

In [20]:
api_url_gdax = api_url.loc["GDAX"].URL

In [21]:
# Create custom authentication for Exchange
class CoinbaseExchangeAuth(AuthBase):
    def __init__(self, exchange):
        self.api_key = api_cred.loc[exchange].KEY
        self.secret_key = api_cred.loc[exchange].SECRET
        self.passphrase = api_cred.loc[exchange].PASSPHRASE

    def __call__(self, request):
        timestamp = str(time.time())
        message = timestamp + request.method + request.path_url + (request.body or '')
        hmac_key = base64.b64decode(self.secret_key)
        signature = hmac.new(hmac_key, message.encode('utf-8'), hashlib.sha256)
        digest = signature.digest()
        signature_b64 = base64.b64encode(digest).decode('utf-8').rstrip('\n')

        request.headers.update({
            'CB-ACCESS-SIGN': signature_b64,
            'CB-ACCESS-TIMESTAMP': timestamp,
            'CB-ACCESS-KEY': self.api_key,
            'CB-ACCESS-PASSPHRASE': self.passphrase,
            'Content-Type': 'application/json'
        })
        return request

In [22]:
auth = CoinbaseExchangeAuth(exchange="GDAX")
auth

In [23]:
# Get accounts
balance_data = requests.get(api_url_gdax + 'accounts', auth=auth)
balance_data = balance_data.json()
balance_data = pd.DataFrame(balance_data)
balance_data

,available,balance,currency,hold,id,profile_id
0,0.0000287411936000,0.0000287411936000,USD,0.0000000000000000,2fb55e27-276a-4f99-b2b7-127729949d1c,6c33d3ef-deba-4eab-8e2c-d6213b4c100d
1,21.2566732700000000,21.2566732700000000,LTC,0.0000000000000000,2f00df30-58fa-42c9-85ff-1c81bd48198e,6c33d3ef-deba-4eab-8e2c-d6213b4c100d
2,1.6841936700000000,1.6841936700000000,ETH,0.0000000000000000,a71932e3-66cd-41d4-bb6f-010e061fb44a,6c33d3ef-deba-4eab-8e2c-d6213b4c100d
3,0.2405942600000000,0.2405942600000000,BTC,0.0000000000000000,1205713f-4d84-46e1-b385-12e268600193,6c33d3ef-deba-4eab-8e2c-d6213b4c100d


In [24]:
for acct in balance_data.id:
    account = 'accounts/'+ acct +'/ledger'
    acct_hist = requests.get(api_url_gdax + account, auth=auth)
    temp = acct_hist.json()
    temp = pd.DataFrame(temp)
    temp.replace({r'\n': ''}, regex=True)
    print(temp)

                  amount                balance                   created_at  \
0    -1.4955134531499000     0.0000287411936000  2017-08-20T19:53:52.215691Z   
1  -498.5044843833000000     1.4955421943435000  2017-08-20T19:53:52.215691Z   
2    -2.4937655197565000   500.0000265776435000  2017-08-20T19:53:33.684975Z   
3  -997.5062079026000000   502.4937920974000000  2017-08-20T19:53:33.684975Z   
4  1500.0000000000000000  1500.0000000000000000  2017-08-18T14:33:02.342129Z   

                                             details         id      type  
0  {'order_id': '4d8bbb55-1d30-4b5d-83e6-4b7b12bf...  248785615       fee  
1  {'order_id': '4d8bbb55-1d30-4b5d-83e6-4b7b12bf...  248785613     match  
2  {'order_id': 'e73e508f-9ebc-4fd7-bf1a-a39edae6...  248785577       fee  
3  {'order_id': 'e73e508f-9ebc-4fd7-bf1a-a39edae6...  248785575     match  
4  {'transfer_id': 'd7b5c42b-c46b-4441-91fc-6aa95...  247140011  transfer  
                amount              balance                  cr

## bitrex code wip

In [25]:
api_url_btrx = api_url.loc["BTRX"].URL
api_url_btrx

'https://bittrex.com/api/v1.1/%s/'

In [ ]:
MARKET_SET = {
    'getopenorders',
    'cancel',
    'sellmarket',
    'selllimit',
    'buymarket',
    'buylimit'
}

ACCOUNT_SET = {
    'getbalances',
    'getbalance',
    'getdepositaddress',
    'withdraw',
    'getorderhistory',
    'getorder',
    'getdeposithistory',
    'getwithdrawalhistory'
}

BUY_ORDERBOOK = 'buy'
SELL_ORDERBOOK = 'sell'
BOTH_ORDERBOOK = 'both'

In [26]:
def using_requests(request_url, apisign):
    return requests.get(
                request_url,
                headers={"apisign": apisign}
            ).json()

In [32]:
class Bittrex(object):
    """
    Used for requesting Bittrex with API key and API secret
    """
    def __init__(self, exchange):
        self.api_key = api_cred.loc[exchange].KEY
        self.secret_key = api_cred.loc[exchange].SECRET
        self.passphrase = api_cred.loc[exchange].PASSPHRASE
#         self.dispatch = dispatch


    def api_query(self, method):
        """
        Queries Bittrex with given method and options
        :param method: Query method for getting info
        :type method: str
        :param options: Extra options for query
        :type options: dict
        :return: JSON response from Bittrex
        :rtype : dict
        """
        nonce = str(int(time.time() * 1000))
        method_set = 'public'

        if method in MARKET_SET:
            method_set = 'market'
        elif method in ACCOUNT_SET:
            method_set = 'account'

        request_url = (api_url % method_set) + method + '?'

        if method_set != 'public':
            request_url += 'apikey=' + self.api_key + "&nonce=" + nonce + '&'

        apisign = hmac.new(self.api_secret.encode(),
                           request_url.encode(),
                           hashlib.sha512).hexdigest()
        return self.dispatch(request_url, apisign)

    def get_markets(self):
        """
        Used to get the open and available trading markets
        at Bittrex along with other meta data.
        :return: Available market info in JSON
        :rtype : dict
        """
        return self.api_query('getmarkets')
    
    def __call__(self, request):
        timestamp = str(time.time())
        message = timestamp + request.method + request.path_url + (request.body or '')
        hmac_key = base64.b64decode(self.secret_key)
        signature = hmac.new(hmac_key, message.encode('utf-8'), hashlib.sha256)
        digest = signature.digest()
        signature_b64 = base64.b64encode(digest).decode('utf-8').rstrip('\n')

        request.headers.update({
            'CB-ACCESS-SIGN': signature_b64,
            'CB-ACCESS-TIMESTAMP': timestamp,
            'CB-ACCESS-KEY': self.api_key,
            'CB-ACCESS-PASSPHRASE': self.passphrase,
            'Content-Type': 'application/json'
        })
        return request

In [31]:
Bittrex.get_balance()

SyntaxError: invalid syntax (<ipython-input-31-e4509fc9fa03>, line 1)

In [35]:
time.time()

1503454030.05098